First we import all the needed modules. Your fbx.pyd should be in your installations site-packages directory. Generating the fbx.pyd is unfortunately very platform specifc and I can't help you with that.

In [7]:
import sys
import os
import math
import fbx
import json

Now we fix up the path to include the FbxCommon module.

In [8]:
sys.path = sys.path + [os.curdir + os.sep + "fbx2020_1"]    

And we need the euclid module for math, along wtith the colours module for RGBA


In [9]:
sys.path = sys.path + [os.curdir + os.sep + "pyeuclid"]

In [10]:
sys.path = sys.path + [os.curdir + os.sep + "colours"]

In [11]:
print(sys.path)

['D:\\Dev\\projects\\python\\pyglsl', 'd:\\apps\\python37\\python37.zip', 'd:\\apps\\python37\\DLLs', 'd:\\apps\\python37\\lib', 'd:\\apps\\python37', '', 'd:\\apps\\python37\\lib\\site-packages', 'd:\\apps\\python37\\lib\\site-packages\\win32', 'd:\\apps\\python37\\lib\\site-packages\\win32\\lib', 'd:\\apps\\python37\\lib\\site-packages\\Pythonwin', 'd:\\apps\\python37\\lib\\site-packages\\IPython\\extensions', 'C:\\Users\\Zaba\\.ipython', '.\\fbx2020_1', '.\\pyeuclid', '.\\colours', '.\\fbx2020_1', '.\\pyeuclid', '.\\colours']


In [12]:
from euclid import Point3
from colours import RGBA
from FbxCommon import *

Next we create the global objects we need to work with : the FbxManager and the FbxScene

In [13]:
def create_sdk_manager():
    result = FbxManager.Create()
    return result

def create_scene(manager, name = "FbxScene"):
    ios = FbxIOSettings.Create(manager, IOSROOT)
    manager.SetIOSettings(ios)
    scene = FbxScene.Create(manager, name)
    return scene

sdk_manager = create_sdk_manager()
fbx_scene = create_scene(sdk_manager)

Next, we load the secne, specifiying which elements we want to import, which in our case is all of them.

In [14]:
def load_scene(manager, scene, file_name):
    importer = FbxImporter.Create(manager, "")    
    result = importer.Initialize(file_name, -1, manager.GetIOSettings())
    if not result:
        return False    
    if importer.IsFBX():
        manager.GetIOSettings().SetBoolProp(EXP_FBX_MATERIAL, True)
        manager.GetIOSettings().SetBoolProp(EXP_FBX_TEXTURE, True)
        manager.GetIOSettings().SetBoolProp(EXP_FBX_EMBEDDED, True)
        manager.GetIOSettings().SetBoolProp(EXP_FBX_SHAPE, True)
        manager.GetIOSettings().SetBoolProp(EXP_FBX_GOBO, True)
        manager.GetIOSettings().SetBoolProp(EXP_FBX_ANIMATION, True)
        manager.GetIOSettings().SetBoolProp(EXP_FBX_GLOBAL_SETTINGS, True)
    
    result = importer.Import(scene)
    importer.Destroy()
    return scene

if load_scene(sdk_manager, fbx_scene, "GenericMan.fbx") == None:
    print("Load Failed")

The scene is organsied as a tree. So we want the root node to begin traversal

In [15]:
 root_node = fbx_scene.GetRootNode()

Now we can walk the tree with the usual recursive jiggery pokey

In [16]:
def walk_nodes_aux(node):
    print(node.GetName())
    for i in range(node.GetChildCount()):
        walk_nodes_aux(node.GetChild(i))
        
def walk_nodes(scene):
    root = scene.GetRootNode()
    walk_nodes_aux(root)
    
walk_nodes(fbx_scene)

RootNode
Character
Root
Waist
Left Hip
Left Knee
Left Foot
Right Hip
Right Knee
Right Foot
Middle Spine
Neck
Left Arm
Left Elbow
Left Hand
Head
Right Arm
Right Elbow
Right Hand


Each node has a transform that is multiplied through the hierarchy and varies over time, with animation. 

In [22]:
def get_node_global_transform(node, time = 0.0):
    t = FbxTime(time)
    return node.EvaluateGlobalTransform(t)

In [23]:
def get_node_local_transform(node, time = 0.0):
    t = FbxTime(time)
    return node.EvaluateLocalTransform(t)

And also, a helper transform, which does not form part of the transform hierarchy which provices an "in-place" offset.

In [24]:
def get_node_geometric_transform(node):
    result = fbx.FbxAMatrix()
    result.SetIdentity()
    if (node and node.GetNodeAttribute()):
        t = node.GetGeometricTranslation(fbx.FbxNode.eSourcePivot)
        r = node.GetGeometricRotation(fbx.FbxNode.eSourcePivot)
        s = node.GetGeometricScaling(fbx.FbxNode.eSourcePivot)
        result = fbx.FbxAMatrix(t,r,s)
    return result

Taken all together, they can calculate the nodes transform.

In [25]:
def get_node_transform(node, time = 0.0):
    result = get_node_global_transform(node, time)
    if (node.GetNodeAttribute()):
        gt = get_node_geometric_transform(node)
        result = result * gt
    return result

In [26]:
transform = get_node_transform(root_node, 0.0)
print(transform)

Not a very useful representation. So we need to write a function to print it.

In [27]:
def print_matrix(m):
    for i in range(0,3):
        print("|", m[i][0], ",", m[i][1], ",", m[i][2], ",", m[i][3], "|") 
    return

print_matrix(transform)

| 1.0 , 0.0 , 0.0 , 0.0 |
| 0.0 , 1.0 , 0.0 , 0.0 |
| 0.0 , 0.0 , 1.0 , 0.0 |


We can work out a node local transform if we have the global transform of parent and child.

In [28]:
def get_node_local_transform(node, time = 0.0):
    transform = get_node_transform(node, time)
    parent = node.GetParent()
    if parent:
        parent_transform = get_node_transform(parent, time)
        inverse_parent_transform = parent_transform.Inverse()
        transform = transform * inverse_parent_transform
    return transform

In [29]:
local_transform = get_node_local_transform(root_node, 0.0)
print_matrix(local_transform)

| 1.0 , 0.0 , 0.0 , 0.0 |
| 0.0 , 1.0 , 0.0 , 0.0 |
| 0.0 , 0.0 , 1.0 , 0.0 |


Thats not so useful, so let's look at one of the childern

In [30]:
child_node = root_node.GetChild(1)
child_transform = get_node_local_transform(child_node, 0.0)
print_matrix(child_transform)

| 1.0 , 0.0 , 0.0 , 0.0 |
| 0.0 , 1.0 , 0.0 , 0.0 |
| 0.0 , 0.0 , 1.0 , 0.0 |


Each node has an attribute which describes which type of data is associated with it

In [31]:
print(root_node.GetNodeAttribute())
print(child_node.GetNodeAttribute())
print(root_node.GetChild(0).GetNodeAttribute())

None


Every fbx type has a classId which we can use to identify it

In [32]:
fbx.FbxMesh.ClassId.GetName()
print(child_node.GetNodeAttribute().ClassId.GetName())

FbxSkeleton


So now we can dump a lot more useful information for each node, and we can have a custom handler function for each node. This is the backbone of an exporter.

In [33]:
def process_mesh(node, time = 0.0):
    print("It's a mesh!")
    return

def process_skeleton(node, time = 0.0):
    print("It's a skeleton")
    return
    
fbx_node_methods = {}

fbx_node_methods[fbx.FbxMesh.ClassId.GetName()] = process_mesh
fbx_node_methods[fbx.FbxSkeleton.ClassId.GetName()] = process_skeleton 

def walk_nodes_aux(node, time = 0.0):
    print(node.GetName())
    node_attribute = node.GetNodeAttribute()
    global_transform = get_node_global_transform(node, time)
    geometric_transform = get_node_geometric_transform(node)
    actual_transform = global_transform * geometric_transform
    print_matrix(actual_transform)
    if (node_attribute and node_attribute.ClassId.GetName() in fbx_node_methods):
        fbx_node_methods[node_attribute.ClassId.GetName()](node, time)
    else:
        print("Unknown node type")
    for i in range(node.GetChildCount()):
        walk_nodes_aux(node.GetChild(i))
        
walk_nodes_aux(root_node)

RootNode
| 1.0 , 0.0 , 0.0 , 0.0 |
| 0.0 , 1.0 , 0.0 , 0.0 |
| 0.0 , 0.0 , 1.0 , 0.0 |
Unknown node type
Character
| 1.0 , 0.0 , 0.0 , 0.0 |
| 0.0 , 1.0 , 0.0 , 0.0 |
| 0.0 , 0.0 , 1.0 , 0.0 |
It's a mesh!
Root
| 1.0 , 0.0 , 0.0 , 0.0 |
| 0.0 , 1.0 , 0.0 , 0.0 |
| 0.0 , 0.0 , 1.0 , 0.0 |
It's a skeleton
Waist
| 1.0 , 0.0 , 0.0 , 0.0 |
| 0.0 , 0.978147720798726 , 0.2079116993217334 , 0.0 |
| 0.0 , -0.20791162496672436 , 0.9781473709858831 , 0.0 |
It's a skeleton
Left Hip
| 1.0000331401824951 , 0.0 , 0.0 , 0.0 |
| 0.0 , 0.6427872521177278 , -0.7660440428796939 , 0.0 |
| 0.0 , 0.7660587909941312 , 0.6427996272546944 , 0.0 |
It's a skeleton
Left Knee
| 1.0000349283810976 , 0.0 , 0.0 , 0.0 |
| 0.0 , 0.5150381899289473 , 0.8571677183484778 , 0.0 |
| 0.0 , -0.8571845273564864 , 0.515048289797239 , 0.0 |
It's a skeleton
Left Foot
| 1.0000349283810976 , 0.0 , 0.0 , 0.0 |
| 0.0 , 0.5150381899289473 , 0.8571677183484778 , 0.0 |
| 0.0 , -0.8571845273564864 , 0.515048289797239 , 0.0 |
It's a skelet

Now we need to process a mesh. A mesh is organised as a number of layers, each carrying different information.

In [34]:
test_mesh_node = root_node.GetChild(0)
test_mesh = test_mesh_node.GetNodeAttribute()

def parse_material(material):
    if material:
        if (material.GetReferenceMode() == FbxLayerElement.eDirect):
            print("Material is directly indexed")
        else:
            print("Material is indirectly indexed")
            index_array = material.GetIndexArray()
            index_array_count = index_array.GetCount()
            print("There are %d indices" % index_array_count)
    else:
        print("No material")
    
def parse_layers(node, mesh):
    print("Mesh has %d layers and %d materials " % (mesh.GetLayerCount(), node.GetMaterialCount()))
    material_count = node.GetMaterialCount()
    layer_count = mesh.GetLayerCount()
    for l in range(layer_count):
            print("In Layer %d " % (l))
            material = mesh.GetLayer(l).GetMaterials()
            parse_material(material)
            
parse_layers(test_mesh_node,test_mesh)
                

Mesh has 2 layers and 1 materials 
In Layer 0 
Material is indirectly indexed
There are 1300 indices
In Layer 1 
No material


A simpler helper function to get a per vertex attribute

In [35]:
    def get_vertex_element(self, vertexIndex):
        mapping_mode = self.elements.GetMappingMode()
        reference_mode = self.elements.GetReferenceMode()  
        if (mapping_mode == FbxLayerElement.eByControlPoint):
            if (reference_mode == FbxLayerElement.eDirect):
                return self.elements.GetDirectArray().GetAt(vertexIndex)
            elif (reference_mode == FbxLayerElement.eIndexToDirect):
                ix = self.elements.GetIndexArray().GetAt(vertexIndex)
                return self.elements.GetDirectArray().GetAt(ix)
        elif (mapping_mode == FbxLayerElement.eByPolygonVertex):
            pass
        elif (mapping_mode == FbxLayerElement.eByPolygon):
            pass
        elif (mapping_mode == FbxLayerElement.eByEdge):
            pass
        elif (mapping_mode == FbxLayerElement.eAllSame):
            return self.elements.GetDirectArray().GetAt(vertexIndex)
        return None

A simple helper function to get a per-polygon attribute

In [36]:
   def get_polygon_element(self, polygonIndex):
        mapping_mode = self.elements.GetMappingMode()
        reference_mode = self.elements.GetReferenceMode()  
        if (mapping_mode == FbxLayerElement.eByControlPoint):
            if (reference_mode == FbxLayerElement.eDirect):
                return self.elements.GetDirectArray().GetAt(plygonIndex)
            elif (reference_mode == FbxLayerElement.eIndexToDirect):
                ix = self.elements.GetIndexArray().GetAt(polygonIndex)
                return self.elements.GetDirectArray().GetAt(ix)
        elif (mapping_mode == FbxLayerElement.eByPolygonVertex):
            pass
        elif (mapping_mode == FbxLayerElement.eByPolygon):
            if (reference_mode == FbxLayerElement.eDirect):
                return self.elements.GetDirectArray().GetAt(plygonIndex)
            elif (reference_mode == FbxLayerElement.eIndexToDirect):
                ix = self.elements.GetIndexArray().GetAt(polygonIndex)
                return self.elements.GetDirectArray().GetAt(ix)    
        elif (mapping_mode == FbxLayerElement.eByEdge):
            pass
        elif (mapping_mode == FbxLayerElement.eAllSame):
            if (reference_mode == FbxLayerElement.eDirect):
                direct_array = self.elements.GetDirectArray()
                return direct_array.GetAt(0)
            elif (reference_mode == FbxLayerElement.eIndexToDirect):
                index_array = self.elements.GetIndexArray()
                ix = index_array.GetAt(0)
                return self.elements.GetDirectArray().GetAt(ix)
        return None

But what are these access modes? When you have an array of values, like UV's for example - they can be stored that way - as an array. Or they can be stored as an array of values and an array of indices which you traverse to get the values. They can also be stored per vertex (ControlPoint) or per Polygon (Face) depending what they are: or ever per Vertex, per Polygon.

In practice these are the main sensible ways to store such an array, but there are a few others. This class addresses them, by abstracting away the differences and pretending everything is a per vertex, per poly attribute.


In [54]:
from euclid import Vector2
from euclid import Vector3

class LayerElement():
    def __init__(self, mesh, layer_index, layer_kind):
        self.mesh = mesh
        self.layer = mesh.GetLayer(layer_index)
        self.elements = self.layer.GetLayerElementOfType(layer_kind)
    def valid(self):
        return (self.layer) and (self.elements)
    def indexed(self):
        return (self.elements.GetReferenceMode() != FbxLayerElement.eDirect)
    def get_element(self, polygonIndex, vertexIndex):
        assert(self.mesh.GetPolygonCount() > polygonIndex)
        assert(self.mesh.GetPolygonSize(polygonIndex) > vertexIndex)
        control_point_index = self.mesh.GetPolygonVertex(polygonIndex, vertexIndex)
        mapping_mode = self.elements.GetMappingMode()
        reference_mode = self.elements.GetReferenceMode() 
        if (mapping_mode == FbxLayerElement.eByControlPoint):
            if (reference_mode == FbxLayerElement.eDirect):
                return self.elements.GetDirectArray().GetAt(control_point_index)
            elif (reference_mode == FbxLayerElement.eIndexToDirect):
                index_array = elements.GetIndexArray()
                assert(index_array.GetCount() == mesh.GetControlPointsCount())
                assert(control_point_index < index_array.GetCount())
                ix = index_array.GetAt(control_point_index)
                direct_array = self.elements.GetDirectArray()
                assert(ix < direct_array.GetCount())
                return self.elements(ix)
        elif (mapping_mode == FbxLayerElement.eByPolygonVertex):
            vertexId = polygonIndex * 3 + vertexIndex;
            if (reference_mode == FbxLayerElement.eDirect):
                direct_array = self.elements.GetDirectArray()
                assert(vertexId < direct_array.GetCount())
                return direct_array.GetAt(vertexId)      
            elif (reference_mode == FbxLayerElement.eIndexToDirect):
                index_array = self.elements.GetIndexArray()
                assert(vertexId < index_array.GetCount())
                ix = index_array.GetAt(vertexId)
                return self.elements.GetDirectArray().GetAt(ix)
        elif (mapping_mode == FbxLayerElement.eByPolygon):
            if (reference_mode == FbxLayerElement.eDirect):
                direct_array = self.elements.GetDirectArray()
                assert(polygonIndex < direct_array.GetCount())
                return direct_array.GetAt(polygonIndex)
            elif (reference_mode == FbxLayerElement.eIndexToDirect):
                index_array = self.elements.GetIndexArray()
                assert(polygonIndex < index_array.GetCount())
                ix = index_array.GetAt(polygonIndex)
                return self.elements.GetDirectArray().GetAt(ix)
        elif (mapping_mode == FbxLayerElement.eAllSame):
            if (reference_mode == FbxLayerElement.eDirect):
                direct_array = self.elements.GetDirectArray()
                return direct_array.GetAt(0)
            elif (reference_mode == FbxLayerElement.eIndexToDirect):
                index_array = self.elements.GetIndexArray()
                ix = index_array.GetAt(0)
                return self.elements.GetDirectArray().GetAt(ix)
        else:
            print("Error - unhandled mapping mode!")
        return None           
    def dump_elements(self, description):
        element_values = []
        polygon_count = self.mesh.GetPolygonCount()
        for pi in range(polygon_count):
            vertex_count = self.mesh.GetPolygonSize(pi)
            polygon_values = []
            for vi in range(vertex_count):
                value = self.get_element(pi, vi)
                value = Vector2(value[0],value[1]) if (type(value) is fbx.FbxVector2) else value
                value = Vector3(value[0], value[1], value[2]) if (type(value) is fbx.FbxVector4) else value
                polygon_values += [ value ]
            element_values += [tuple(polygon_values) ]
        print (description, element_values)
            

Also a pair of helper functions to show us the mapping mode (element frequency) and reference mode (whether it's indexed or not)

In [55]:
def parse_mapping_mode(mode):
    mapping_types = [ "None", "By Control Point", "By Polygon and Control Point", "By Polygon", "By Edge", "All Same" ]
    return mapping_types[int(mode)]

In [56]:
def parse_reference_mode(mode):
    reference_types = [ "None", "Direct", "IndexToDirect"]
    return reference_types[int(mode)]

With all that out of the way, we can start examining the data on our mesh.

In [57]:
def parse_control_points(node, mesh):
    # vertices
    control_points_count = mesh.GetControlPointsCount()
    control_points = mesh.GetControlPoints()
    print(f"Mesh has {control_points_count} control points")
    layer_count = mesh.GetLayerCount()
    # normals
    for i in range(layer_count):
        print(f"Layer {i}")
        normals = mesh.GetLayer(i).GetNormals()
        if normals:
            mapping_mode = normals.GetMappingMode()
            reference_mode = normals.GetReferenceMode()
            print("Reference mode %s" % parse_reference_mode(reference_mode))
            print("Mapping mode %s" % parse_mapping_mode(mapping_mode))    
        else:
            print("Layer %d has no normals" % (i))
 
parse_control_points(test_mesh_node, test_mesh)

Mesh has 1314 control points
Layer 0
Reference mode IndexToDirect
Mapping mode By Polygon and Control Point
Layer 1
Layer 1 has no normals


Now we know how our data is structured, we can walk over it, poly by poly and vertex by vertex to get our data. We are *not* going to assume the vertices are triangles. If that is important, the sdk has a triangulate function.

In [58]:
from euclid import Point3 
v = test_mesh.GetControlPointAt(0)
vertex = Point3(v[0], v[1], v[2]) 

Now we know how to get a point, we can dump the values

In [59]:
def process_mesh(node, mesh):
    vertices = []
    indices = []
    vertex_count = mesh.GetControlPointsCount()
    for vi in range(0, vertex_count):
        v = mesh.GetControlPointAt(vi)
        vertices += [ Point3(v[0], v[1], v[2]) ]
    poly_count = mesh.GetPolygonCount()
    for pi in range(0, poly_count):
        face_indices = []
        for vi in range(0, mesh.GetPolygonSize(pi)):
            i = mesh.GetPolygonVertex(pi, vi)
            face_indices += [ i ]
        indices += [ tuple(face_indices) ]
    return (vertices, indices)
            
process_mesh(test_mesh_node, test_mesh)

([Point3(-41.00, 57.40, 26.75),
  Point3(-41.00, 90.73, 26.75),
  Point3(0.00, 90.73, 25.69),
  Point3(0.00, 62.61, 14.79),
  Point3(-41.00, 124.07, 26.75),
  Point3(0.00, 124.07, 26.75),
  Point3(-41.00, 157.40, 26.75),
  Point3(0.00, 152.19, 18.39),
  Point3(14.98, 90.47, 24.39),
  Point3(9.58, 55.80, 17.83),
  Point3(15.61, 122.92, 24.98),
  Point3(14.25, 151.93, 17.48),
  Point3(36.66, 89.32, 11.90),
  Point3(35.70, 55.06, 11.60),
  Point3(41.11, 117.50, 13.83),
  Point3(30.80, 150.62, 13.42),
  Point3(35.70, 55.06, -11.60),
  Point3(36.66, 89.32, -11.90),
  Point3(14.98, 90.47, -24.39),
  Point3(9.58, 55.80, -17.83),
  Point3(41.11, 117.50, -13.83),
  Point3(15.61, 122.92, -24.98),
  Point3(30.80, 150.62, -13.42),
  Point3(14.25, 151.93, -17.48),
  Point3(0.00, 90.73, -25.69),
  Point3(0.00, 62.61, -14.79),
  Point3(0.00, 124.07, -26.75),
  Point3(0.00, 152.19, -18.39),
  Point3(-41.00, 90.73, -26.75),
  Point3(-41.00, 57.40, -26.75),
  Point3(-41.00, 124.07, -26.75),
  Point3(-41

To get the *other* vertex attributes, such as normals, UVs and vertex colors, we need to walk the layers.

In [60]:
def list_layers(mesh):
    layer_count = mesh.GetLayerCount()
    print(f"Mesh has {layer_count} layers.")
    element_types = { FbxLayerElement.eNormal : "Normals", 
                      FbxLayerElement.eUV : "UVs", 
                      FbxLayerElement.eVertexColor : "VertexColor" }
    for i in range(layer_count):
        print(f"Layer {i}")
        for element_type in element_types:         
            elements = LayerElement(mesh, i, element_type)
            element_kind = element_types[element_type]
            if elements.valid():   
                print(f"Layer {i} has {element_kind} elements")
                elements.dump_elements(element_kind)
            else:
                print(f"Layer {i} has no {element_kind} elements")

list_layers(test_mesh)

Mesh has 2 layers.
Layer 0
Layer 0 has Normals elements
Normals [(Vector3(0.59, -0.66, -0.35), Vector3(0.48, -0.63, -0.54), Vector3(0.67, -0.21, -0.65), Vector3(0.85, -0.27, -0.37)), (Vector3(0.85, -0.27, -0.37), Vector3(0.59, -0.74, 0.13), Vector3(0.30, -0.94, 0.00), Vector3(0.30, -0.93, -0.15)), (Vector3(0.30, -0.93, -0.15), Vector3(0.63, -0.70, -0.11), Vector3(0.92, -0.12, 0.33), Vector3(0.73, -0.18, 0.63)), (Vector3(0.73, -0.18, 0.63), Vector3(0.67, -0.42, 0.54), Vector3(0.85, -0.37, 0.27), Vector3(0.91, 0.00, -0.35)), (Vector3(0.91, 0.00, -0.35), Vector3(0.92, 0.07, -0.34), Vector3(0.98, 0.00, -0.00), Vector3(0.98, -0.06, -0.01)), (Vector3(0.98, -0.06, -0.01), Vector3(0.31, -0.79, 0.44), Vector3(0.48, -0.76, 0.32), Vector3(0.67, -0.42, 0.54)), (Vector3(0.67, -0.42, 0.54), Vector3(0.43, -0.46, 0.74), Vector3(-0.17, -0.84, 0.42), Vector3(-0.12, -0.98, 0.10)), (Vector3(-0.12, -0.98, 0.10), Vector3(0.03, -0.98, 0.12), Vector3(0.09, -0.82, 0.47), Vector3(-0.24, -0.25, 0.90)), (Vector3(

We now have nearly everything we need to know to build an exporter. However we still don't know anything about materials or textures. Materials are not associated with the mesh, but with the node, so thats where we begin..

In [70]:
test_mesh_node.GetMaterialCount()
print(test_mesh_node)
print(test_mesh.GetNode())

But the actual material information is in the layers, so we iterate through them again, looking for materials

In [72]:
def list_material_layers(mesh):
    layer_count = mesh.GetLayerCount()
    print(f"Mesh has {layer_count} layers.")
    for i in range(layer_count):
        materials = test_mesh.GetLayer(i).GetMaterials()
        if materials:
            print(f"Layer {i} is a material layer")
list_material_layers(test_mesh)

Mesh has 2 layers.
Layer 0 is a material layer


The mapping mode in this case reflects if this is a model with a single material applied to all polygons or a per polygon material.


In [74]:
def list_material_layers(mesh):
    layer_count = mesh.GetLayerCount()
    print(f"Mesh has {layer_count} layers.")
    isAllSame = True
    for i in range(layer_count):
        materials = test_mesh.GetLayer(i).GetMaterials()
        if materials:
            print(f"Layer {i} is a material layer")
            if materials.GetMappingMode() == FbxLayerElement.eAllSame:
                material = mesh.GetNode().GetMaterial(materials.GetIndexArray().GetAt(0))    
                matId = material.GetIndexArray().GetAt(0)
                print(f"On layer {i} all polygons have {matId} applied")        
            if materials.GetMappingMode() == FbxLayerElement.eByPolygon:
                polygonCount = mesh.GetPolygonCount()
                for pi in range(polygonCount):
                    material = mesh.GetNode().GetMaterial(materials.GetIndexArray().GetAt(pi))
                    matId = materials.GetIndexArray().GetAt(pi)

                    print(f"On polygon {pi} material {matId} is applied")

list_material_layers(test_mesh)

Mesh has 2 layers.
Layer 0 is a material layer
On polygon 0 material 0 is applied
On polygon 1 material 0 is applied
On polygon 2 material 0 is applied
On polygon 3 material 0 is applied
On polygon 4 material 0 is applied
On polygon 5 material 0 is applied
On polygon 6 material 0 is applied
On polygon 7 material 0 is applied
On polygon 8 material 0 is applied
On polygon 9 material 0 is applied
On polygon 10 material 0 is applied
On polygon 11 material 0 is applied
On polygon 12 material 0 is applied
On polygon 13 material 0 is applied
On polygon 14 material 0 is applied
On polygon 15 material 0 is applied
On polygon 16 material 0 is applied
On polygon 17 material 0 is applied
On polygon 18 material 0 is applied
On polygon 19 material 0 is applied
On polygon 20 material 0 is applied
On polygon 21 material 0 is applied
On polygon 22 material 0 is applied
On polygon 23 material 0 is applied
On polygon 24 material 0 is applied
On polygon 25 material 0 is applied
On polygon 26 material 0 is

Finally the last part of the puzzle : textures - we know the material so we can find the texture

In [79]:
def list_material_texture(material):
    property = material.FindProperty(FbxSurfaceMaterial.sDiffuse)
    textureLayerCount = property.GetSrcObjectCount(FbxCriteria.ObjectType(FbxLayeredTexture.ClassId))
    if textureLayerCount > 0:
        print(f"Layerd textures not supported yet")
    else:
        textureCount = property.GetSrcObjectCount(FbxCriteria.ObjectType(FbxTexture.ClassId))
        for j in range(textureCount):
            texture = property.GetSrcObject(FbxCriteria.ObjectType(FbxTexture.ClassId),j)
            textureName = texture.GetName()
            print(f"Texture {j} : {textureName}")
            
def list_material_layers(mesh):
    layer_count = mesh.GetLayerCount()
    print(f"Mesh has {layer_count} layers.")
    isAllSame = True
    for i in range(layer_count):
        materials = test_mesh.GetLayer(i).GetMaterials()
        if materials:
            print(f"Layer {i} is a material layer")
            if materials.GetMappingMode() == FbxLayerElement.eAllSame:
                material = mesh.GetNode().GetMaterial(materials.GetIndexArray().GetAt(0))    
                matId = material.GetIndexArray().GetAt(0)
                print(f"On layer {i} all polygons have {matId} applied")   
                list_material_texture(material)
            if materials.GetMappingMode() == FbxLayerElement.eByPolygon:
                polygonCount = mesh.GetPolygonCount()
                for pi in range(polygonCount):
                    material = mesh.GetNode().GetMaterial(materials.GetIndexArray().GetAt(pi))
                    matId = materials.GetIndexArray().GetAt(pi)
                    print(f"On polygon {pi} material {matId} is applied")
                    list_material_texture(material)
                    

list_material_layers(test_mesh)

Mesh has 2 layers.
Layer 0 is a material layer
On polygon 0 material 0 is applied
Texture 0 : Skin_tex
On polygon 1 material 0 is applied
Texture 0 : Skin_tex
On polygon 2 material 0 is applied
Texture 0 : Skin_tex
On polygon 3 material 0 is applied
Texture 0 : Skin_tex
On polygon 4 material 0 is applied
Texture 0 : Skin_tex
On polygon 5 material 0 is applied
Texture 0 : Skin_tex
On polygon 6 material 0 is applied
Texture 0 : Skin_tex
On polygon 7 material 0 is applied
Texture 0 : Skin_tex
On polygon 8 material 0 is applied
Texture 0 : Skin_tex
On polygon 9 material 0 is applied
Texture 0 : Skin_tex
On polygon 10 material 0 is applied
Texture 0 : Skin_tex
On polygon 11 material 0 is applied
Texture 0 : Skin_tex
On polygon 12 material 0 is applied
Texture 0 : Skin_tex
On polygon 13 material 0 is applied
Texture 0 : Skin_tex
On polygon 14 material 0 is applied
Texture 0 : Skin_tex
On polygon 15 material 0 is applied
Texture 0 : Skin_tex
On polygon 16 material 0 is applied
Texture 0 : Ski

On polygon 827 material 0 is applied
Texture 0 : Skin_tex
On polygon 828 material 0 is applied
Texture 0 : Skin_tex
On polygon 829 material 0 is applied
Texture 0 : Skin_tex
On polygon 830 material 0 is applied
Texture 0 : Skin_tex
On polygon 831 material 0 is applied
Texture 0 : Skin_tex
On polygon 832 material 0 is applied
Texture 0 : Skin_tex
On polygon 833 material 0 is applied
Texture 0 : Skin_tex
On polygon 834 material 0 is applied
Texture 0 : Skin_tex
On polygon 835 material 0 is applied
Texture 0 : Skin_tex
On polygon 836 material 0 is applied
Texture 0 : Skin_tex
On polygon 837 material 0 is applied
Texture 0 : Skin_tex
On polygon 838 material 0 is applied
Texture 0 : Skin_tex
On polygon 839 material 0 is applied
Texture 0 : Skin_tex
On polygon 840 material 0 is applied
Texture 0 : Skin_tex
On polygon 841 material 0 is applied
Texture 0 : Skin_tex
On polygon 842 material 0 is applied
Texture 0 : Skin_tex
On polygon 843 material 0 is applied
Texture 0 : Skin_tex
On polygon 844

Animations

In [115]:
def get_anim_stacks(scene):
    anim_stack_classid = FbxAnimStack.ClassId
    stack_names = fbx.FbxStringArray()
    scene.FillAnimStackNameArray(stack_names)
    num_stacks = stack_names.GetCount()
    for stack_index in range(0,num_stacks):
        anim_name = stack_names[stack_index]
        if anim_name == None:
            continue
        anim_stack = scene.FindMember(anim_stack_classid, anim_name.Buffer())
        reference_time_span = anim_stack.GetReferenceTimeSpan()
        local_time_span = anim_stack.GetLocalTimeSpan()
        print("Anim Stack ", anim_name)
        print("Reference Start, Stop ", anim_stack.GetReferenceTimeSpan().GetStart().GetSecondDouble(), anim_stack.GetReferenceTimeSpan().GetStop().GetSecondDouble())
        print("Local Start, Stop ", anim_stack.GetLocalTimeSpan().GetStart().GetSecondDouble(), anim_stack.GetLocalTimeSpan().GetStop().GetSecondDouble())
        
get_anim_stacks(fbx_scene)

Anim Stack  Idle
Reference Start, Stop  0.0 1.5000000779478457
Local Start, Stop  0.0 1.5000000779478457
Anim Stack  Walk
Reference Start, Stop  0.0 1.8000000921201813
Local Start, Stop  0.0 1.8000000921201813
Anim Stack  Jump
Reference Start, Stop  0.0 1.8000000921201813
Local Start, Stop  0.0 1.8000000921201813
Anim Stack  Crouch
Reference Start, Stop  0.0 1.8000000921201813
Local Start, Stop  0.0 1.8000000921201813
